# Project 2: Customer Exit Prediction

# Introduction:

*   In this project, you will predict whether a customer will exit (CustomerExit) based on various service and demographic features.
*   You will conduct data exploration, preprocessing, and train machine learning models to predict customer exit.
*   Finally, you'll evaluate the models using classification metrics.

**Datasets Locations and Names:** Canvas -> Modules -> Week 10 -> Dataset -> "customer_exit.csv".

Please save your Google Colab project as FirstName_LastName_Project2.ipynb and submit (upload) your source code to Canvas.

# Dataset Description:

* **ServiceFee:** Monthly fee the customer pays.
* **SubscriptionType:** Type of contract (e.g., month-to-month, yearly).
* **MembershipDuration**: How many months the customer has been with the service.
* **CumulativeCharges:** Total amount billed to the customer.
* **Age:** Age of the customer.
* **Gender:** Gender of the customer.
* **CustomerExit:** The target variable, indicating whether the customer left (1 = exit, 0 = stays).

# Task 1: Exploratory Data Analysis (EDA)
* Visualize the distribution of the target variable (`CustomerExit`).
* Create scatter plots or histograms to explore relationships between features.
* Check for missing values.

In [ ]:
# Your EDA code here

# Task 2: Data Preprocessing with Pipelines and Feature Extraction
* Use Pipeline for handling missing values, scaling, and feature extraction.
* Use ColumnTransformer for applying different transformations to numeric and categorical features.

In [ ]:
# Your data preprocessing code here

# Task 3: Train Classification Models
Train at least 5 classification models within a pipeline:

In [ ]:
# Your model training code here

# Task 4: Model Evaluation
* Evaluate your models using k-fold cross-validation (at least 5 folds) to assess how well the models generalize.
* Calculate and report the average accuracy, precision, recall, specificity, and F1-score from the cross-validation.
* Compare and discuss models based on their cross-validated performance.


In [ ]:
# Your model evaluation code here

# Task 5: Hyperparameter Tuning with GridSearchCV
* Apply GridSearchCV to your best model for hyperparameter tuning.

In [ ]:
# Your GridSearchCV code here

# Conclusion:
* Summarize your findings and explain any interesting insights from your analysis.